In [2]:
import pandas as pd
import numpy as np
import xarray as xr
import cdsapi 

# viz
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
from cartopy.io import shapereader
import plotly.graph_objects as go
import plotly.express as px
import folium
import seaborn as sns

import glob
import sys
import os

from scipy.interpolate import griddata
from numpy.polynomial.polynomial import Polynomial

In [4]:
import geopandas as gpd
import rioxarray
from shapely.geometry import mapping

# functions for state masking

def get_shape_us(shapefile, state_abbreviation):
    shapefile = shapefile.to_crs("EPSG:4326")
    state_shape = shapefile[shapefile.STUSPS == state_abbreviation].geometry
    return state_shape

def mask_xr_dataset(xr_data, shape):
    xr_data = xr_data.assign_coords(longitude=(((xr_data.longitude + 180) % 360) - 180)).sortby('longitude')
    xr_data.rio.write_crs("EPSG:4326", inplace=True)
    xr_data_masked = xr_data.rio.clip(shape.geometry.apply(mapping), shape.crs)
    return xr_data_masked

In [33]:
from numpy.polynomial import Polynomial

# function to add trend line

def fit_polynomial(df):
    # add trend line
    p = Polynomial.fit(df.index, df['tp'], 1)
    print('Slope: ', p.convert().coef[1])
    x_values = np.linspace(df.index.min(), df.index.max(), len(df.index))
    y_values = p(x_values)
    df['trend'] = y_values
    df = df[['year', 'tp', 'trend']]
    return df

In [28]:
################################################
# MDK Final Frequency Data preprocessing
################################################

ERA5 = xr.open_mfdataset('../../../data/UNSEEN/hurricane/ERA5_hourly/ERA5_????.nc', combine='by_coords')
lat_era5 = ERA5.latitude.values
lon_era5 = ERA5.longitude.values

LSMask = xr.open_dataset('../../../data/UNSEEN/hurricane/ERA5_land_sea_mask_new.nc')
LSMask['longitude'] = (((LSMask['longitude'] + 180) % 360) - 180)
LSMask_interp = LSMask.interp(latitude=lat_era5, longitude=lon_era5)

# uncomment below lines to filter for a specific state

us_shapefile = gpd.read_file('../../data/shapefiles/us/tl_2023_us_state.shp')
shape = get_shape_us(us_shapefile, 'MS')
filtered_era5 = mask_xr_dataset(ERA5, shape)

ERA5_land = (
    ERA5.where(LSMask_interp['lsm'].sel(time = '1981').squeeze('time') > 0.5)
)

# area_weights = np.cos(np.deg2rad(filtered_era5.latitude))
# area_weights = area_weights.expand_dims({'longitude': filtered_era5.longitude})

#filtered_era5['tp'] = filtered_era5['tp'] * area_weights

ERA5_coarsened = ERA5_land.coarsen(latitude=3, longitude=3, boundary='trim').sum()
ERA5_agg = ERA5_coarsened.resample(time='1D').sum()

ERA5_agg_df = ERA5_agg.to_dataframe().reset_index()
ERA5_agg_df = ERA5_agg_df.loc[ERA5_agg_df['time'].dt.month.between(6, 10)]
ERA5_agg_df = ERA5_agg_df[['time', 'tp']]
#ERA5_agg_df.to_csv('../../../data/UNSEEN/hurricane/rainfall_preprocessed/ERA5_1d_raw.csv', index=False)


In [34]:

# data preprocessing for data viz of 999th percentile count data

ERA5_agg = ERA5_agg.chunk({'time': -1})
extreme_precipitation_treshold = ERA5_agg.sel(time=slice(None, '2000')).tp.quantile(.999, dim=['time'])
count_above_threshold = xr.where(ERA5_agg['tp'] > extreme_precipitation_treshold, 1, 0)
ERA5_agg_annual = count_above_threshold.resample(time='YE').sum()
ERA5_agg_annual_total = ERA5_agg_annual.sum(['latitude', 'longitude'])
ERA5_extreme_frequency_df = ERA5_agg_annual_total.to_dataframe().reset_index()
ERA5_extreme_frequency_df['year'] = ERA5_extreme_frequency_df['time'].dt.year
ERA5_extreme_frequency_df = ERA5_extreme_frequency_df[ERA5_extreme_frequency_df['year'] != 2024]
plot_df = fit_polynomial(ERA5_extreme_frequency_df)
plot_df.to_csv('../../../data/UNSEEN/hurricane/rainfall_preprocessed/ERA5_999_counts.csv', index=False)




Slope:  1.2129266082754457


In [24]:
# data preprocessing for state level frequency analysis for damages

state_abbr = 'MS'

def count_extreme_rainfall_events(xr_data, percentile):
    era5_5d = xr_data.resample(time='1D').sum()
    era5_5d = era5_5d.chunk({'time': -1})
    extreme_precipitation_treshold = era5_5d.sel(time=slice(None, '2000')).tp.quantile(percentile, dim=['time'])
    count_above_threshold = xr.where(era5_5d['tp'] > extreme_precipitation_treshold, 1, 0)
    count_above_threshold = count_above_threshold.sel(time=count_above_threshold['time'].dt.month.isin([6, 7, 8, 9, 10]))
    count_above_threshold_df = count_above_threshold.to_dataframe().reset_index()
    return count_above_threshold_df

ERA5 = xr.open_mfdataset('../../../data/UNSEEN/hurricane/ERA5_hourly/ERA5_????.nc', combine='by_coords')
# lat_era5 = ERA5.latitude.values
# lon_era5 = ERA5.longitude.values

# LSMask = xr.open_dataset('../../../data/UNSEEN/hurricane/ERA5_land_sea_mask_new.nc')
# LSMask['longitude'] = (((LSMask['longitude'] + 180) % 360) - 180)
# LSMask_interp = LSMask.interp(latitude=lat_era5, longitude=lon_era5)

# uncomment below lines to filter for a specific state

us_shapefile = gpd.read_file('../../../data/shapefiles/us/tl_2023_us_state.shp')
shape = get_shape_us(us_shapefile, state_abbr)
filtered_era5 = mask_xr_dataset(ERA5, shape)

# ERA5_land = (
#     ERA5.where(LSMask_interp['lsm'].sel(time = '1981').squeeze('time') > 0.5)
# )

# area_weights = np.cos(np.deg2rad(filtered_era5.latitude))
# area_weights = area_weights.expand_dims({'longitude': filtered_era5.longitude})

#filtered_era5['tp'] = filtered_era5['tp'] * area_weights

ERA5_coarsened = filtered_era5.coarsen(latitude=3, longitude=3, boundary='trim').sum()
ERA5_extreme_count = count_extreme_rainfall_events(ERA5_coarsened, 0.999)
ERA5_extreme_count['year'] = ERA5_extreme_count['time'].dt.year
ERA5_extreme_count_grouped = ERA5_extreme_count.groupby('year')['tp'].sum().reset_index(name='tp')
print(ERA5_extreme_count_grouped.head())
ERA5_extreme_count_grouped.to_csv(f'../../../data/UNSEEN/hurricane/rainfall_preprocessed/states/ERA5_999_counts_{state_abbr}.csv', index=False)

   year  tp
0  1981   0
1  1982   1
2  1983   0
3  1984   3
4  1985   6
